In [4]:
!pip install flask
!pip install numpy
!pip install tensorflow
!pip install flask-socketio
!pip install "python-socketio[asyncio_client]"

  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
  Using cached absl_py-0.13.0-py3-none-any.whl (132 kB)
  Using cached wrapt-1.12.1.tar.gz (27 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached six-1.15.0-py2.py3-none-any.whl (10 kB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached typing_extensions-3.7.4.3-py3-none-any.whl (22 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached Markdown-3.3.4-py3-none-any.whl (97 kB)
  Using cached tensorboard_plugin_wit-1.8.0-py3-none-any.whl (781 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-an

In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

from flask import Flask, render_template
from flask_socketio import SocketIO, send, emit
import socketio

import time
from typing import List

from requests import get

In [ ]:
sio = socketio.AsyncClient()
app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
socketio = SocketIO(app)

computeNodes = [str]
dispatchIP = get('https://api.ipify.org').text

In [ ]:
def partition(model: tf.keras.Model, partitions: List[int]) -> tf.keras.Model:
    #To slice properly, add the first and last layers to the partition list
    partitions = [0].append(partitions)
    partitions = partitions.append(len(model.layers) - 1)
    parts = [tf.keras.Model.layers]
    for i in range(1, len(partitions)):
        part = model.layers[partitions[i-1]:partitions[i]]
        parts.append(part)
    
    models = [tf.keras.Model]
    for p in range(len(parts)):
        if p == 0:
            inpt = model.input
        else:
            inpt = keras.Input(tensor=models[p-1].output)
        models.append(
            keras.Model(
                inputs=inpt,
                outputs=[layer.output for layer in parts[p]]
            )
        )
    return models

def dispatchModels(client: socketio.AsyncClient, models: List[tf.keras.Model], nodeIPs: List[str]) -> None:
    for i in range(len(models)):
        model_json = models[i].to_json()
        client.connect(nodeIPs[i], namespaces=['/recv_model'], auth={"name": "dispatcher"})
        if i != len(models) - 1:
            nextNode = nodeIPs[i + 1]
        else:
            # Reached the end of the nodes, the last node needs to point back to the dispatcher
            nextNode = dispatchIP
        client.emit("dispatch", data={"model": model_json, "nextNode": nextNode}, namespace=['/recv_model'])
        client.disconnect()
    
    client.connect(nodeIPs[0], namespaces=['/recv_data'], auth={"name": "dispatcher"})

def startDistEdgeInference(client: socketio.AsyncClient, model_input: tf.Tensor):
    client.emit("data", data=model_input.numpy().tolist(), namespaces=['/recv_data'], auth={"name": "dispatcher"})

In [ ]:
in_list = [[1, 2], [3, 4], [5, 6]]
inpt = tf.convert_to_tensor(in_list)

layer_a = tf.keras.layers.Dense(3, kernel_initializer=tf.constant_initializer(1.))
layer_b = tf.keras.layers.Dense(1, kernel_initializer=tf.constant_initializer(1.))
layer_c = tf.keras.layers.Dense(2, kernel_initializer=tf.constant_initializer(1.))

In [ ]:
model = tf.keras.Sequential()
model.add(layer_a)
model.add(layer_b)
model.add(layer_c)
out = model(inpt)
print("Local run: {}".format(out))
models_to_dispatch = partition(model, [1])

In [ ]:
@socketio.on('from node', namespace="/net")
def handle_data(node, data):
    print("Data handled")
    if node == "part1":
        print("Sending to part2")
        socketio.emit('to part2', {"server": data}, namespace="/net")
    elif node == "part2":
        print("Done distributing, result is {}".format(data))
        exit(0)

@socketio.on('connect', namespace="/recv_model")
def client_connect(auth):
    print("Auth: ", auth)
    print("{} has connected".format(auth["name"]))
    if auth["name"] == "part2":
        print("Emitting to part1")
        socketio.emit('to part1', {"server": in_list}, namespace="/net")

In [ ]:
if __name__ == '__main__':
    dispatchModels(sio, models_to_dispatch, computeNodes)
    startDistEdgeInference(inpt)
    socketio.run(app, port=5000)